In [1]:
#install packages
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
connection_string = ['connectionString']
CONNECTION_STRING_AZURE_STORAGE = connection_string
container_azure = 'housingsales'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(container_azure)

water_toxicity_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.head())
    # If I have only one csv, I am doing the following instructions
    housingsales_df = df.copy()


housingsales.csv
  state  property_zip5 property_street_address   property_city  \
0    AZ        85143.0       1 E CORAL BEAN DR  SAN TAN VALLEY   
1    AZ        85143.0      1 E PEPPERGRASS PL  SAN TAN VALLEY   
2    AZ        85173.0         1 N MESQUITE DR        SUPERIOR   
3    AZ        85143.0      1 W CANYON ROCK RD  SAN TAN VALLEY   
4    AZ        85143.0        1 W MILL REEF DR  SAN TAN VALLEY   

  property_county property_id        sale_datetime property_type  sale_price  \
0           PINAL   210572230  2021-04-20 00:00:00   RESIDENTIAL         0.0   
1           PINAL   210571490  2020-09-16 00:00:00   RESIDENTIAL         0.0   
2           PINAL   10526004A  2021-02-25 00:00:00           NaN         0.0   
3           PINAL   210702840  2011-09-01 00:00:00   RESIDENTIAL     90000.0   
4           PINAL   210591110  2009-08-14 00:00:00   RESIDENTIAL     65000.0   

  seller_1_name  ... land_assessed_date  seller_1_state  seller_2_state  \
0           NaN  ...          

/var/folders/z_/vwk5bc2d7jxd_xbknt7zmjx40000gn/T/ipykernel_67462/134740007.py:20: DtypeWarning: Columns (17,36,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


In [5]:
housingsales_df = housingsales_df[housingsales_df['sale_price'] != 0.0]
# Assuming 'housingsales_df' is your pandas DataFrame
housingsales_df['sale_price'] = housingsales_df['sale_price'].astype('float')  # Convert to float if it's not already

# If 'sale_price' represents currency values and you want to represent them as 'money' data type
housingsales_df['sale_price'] = housingsales_df['sale_price'].astype('float').map('${:,.2f}'.format)

# Now 'sale_price' column will be formatted as currency (e.g., $1,234.56)
housingsales_df.head()

,state,property_zip5,property_street_address,property_city,property_county,property_id,sale_datetime,property_type,sale_price,seller_1_name,...,land_assessed_date,seller_1_state,seller_2_state,buyer_1_state,buyer_2_state,total_assessed_value,total_appraised_value,land_appraised_value,building_appraised_value,land_type
3,AZ,85143.0,1 W CANYON ROCK RD,SAN TAN VALLEY,PINAL,210702840,2011-09-01 00:00:00,RESIDENTIAL,"$90,000.00",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZ,85143.0,1 W MILL REEF DR,SAN TAN VALLEY,PINAL,210591110,2009-08-14 00:00:00,RESIDENTIAL,"$65,000.00",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AZ,85623.0,10 N BACHMAN WASH RD,ORACLE,PINAL,30823026B,2009-07-21 00:00:00,RESIDENTIAL,"$294,254.00",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AZ,85132.0,10 N CENTRAL AVE,FLORENCE,PINAL,200440810,2003-11-04 00:00:00,RESIDENTIAL,"$98,000.00",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,AZ,85138.0,10 N CHERRY LN,MARICOPA,PINAL,501290850,2000-07-17 00:00:00,MOBILE HOME,"$12,500.00",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
